In [1]:
import json

In [14]:
f=open('openalexco.json')
co=json.load(f)
f.close()

In [43]:
def filter_concept(concepts,concept='Physics'):
    return any([True for c in concepts if c.get('display_name')==concept])
def filter_ror(authorships,ror="https://ror.org/03bp5hc83"):
  return any([any([i.get('ror')=="https://ror.org/03bp5hc83" for i in a.get('institutions')])  for a in authorships])

In [39]:
ph=[d for d in co if filter_concept(d.get('concepts'),concept='Physics') ]
del(co)

In [49]:
len(ph)

17021

In [247]:
pp    = [d for d in ph if (filter_concept(d.get('concepts'),concept='Particle physics') or 
                           filter_concept(d.get('concepts'),concept='Mathematical physics') or 
                           filter_concept(d.get('concepts'),concept='Theoretical physics')
                          ) ]
not_pp= [d for d in ph if not (filter_concept(d.get('concepts'),concept='Particle physics') or 
                           filter_concept(d.get('concepts'),concept='Mathematical physics') or 
                           filter_concept(d.get('concepts'),concept='Theoretical physics')
                          ) ]
# Physics → 'Astrophysics'    → 'Dark matter'
#         → 'Nuclear physics' → ['Large Hadron Collider', * (particle physics)']

In [248]:
len(pp)

2525

In [249]:
ph_udea     = [d for d in ph     if filter_ror(d.get('authorships'),ror="https://ror.org/03bp5hc83")]
pp_udea     = [d for d in pp     if filter_ror(d.get('authorships'),ror="https://ror.org/03bp5hc83")]
not_pp_udea = [d for d in not_pp if filter_ror(d.get('authorships'),ror="https://ror.org/03bp5hc83")]

In [250]:
len(ph_udea),len(pp_udea),len(not_pp_udea)+len(pp_udea)

(1886, 340, 1886)

There are 491 in inspirehep, 

In [251]:
import pandas as pd

In [252]:
df=pd.DataFrame(pp_udea)

In [253]:
def get_ror_authors(authorships,ror="https://ror.org/03bp5hc83"):
    authorships_udea=[]
    for a in authorships:
        a_udea=False
        for i in a.get('institutions'):
            if i.get('ror')==ror:
                a_udea=True
        if a_udea:
            authorships_udea.append(a)
    return authorships_udea

In [254]:
df['authorships_udea']=df['authorships'].apply(lambda authorships: get_ror_authors(authorships,ror="https://ror.org/03bp5hc83"))

In [255]:
pp_udea_au=pd.DataFrame([item for subitem in df['authorships_udea'].to_list() for item in subitem])

TODO: combine with full names

In [256]:
pp_udea_au['id']=pp_udea_au['author'].str['id']
pp_udea_au['display_name']=pp_udea_au['author'].str['display_name']

In [257]:
d=pp_udea_au['id'].value_counts().to_dict()

In [258]:
pp_udea_au['works_count']=pp_udea_au['id'].apply(lambda k: d[k])

In [748]:
pp_udea_au_unique=pp_udea_au.drop_duplicates(subset='id').sort_values('works_count',ascending=False)
pp_udea_au_unique[:1]

,author_position,author,institutions,raw_affiliation_string,id,display_name,works_count
36,middle,"{'id': 'https://openalex.org/A2998343963', 'display_name': 'J. D. Ruiz Alvarez', 'orcid': 'https://orcid.org/0000-0002-3306-0363'}","[{'id': 'https://openalex.org/I35961687', 'display_name': 'University of Antioquia', 'ror': 'https://ror.org/03bp5hc83', 'country_code': 'CO', 'type': 'education'}]",None,https://openalex.org/A2998343963,J. D. Ruiz Alvarez,107


In [749]:
def split_names(s):
    '''
    TODO implement: name other lastname  → Diego Alejandro Restrepo
    '''
    import re
    s=re.sub('(\w)\.\s*(\w)\.*',r'\1 \2',s).strip().replace('-',' ')
    ss=s.split(' ')
    if not ss:
        return {}
    if len(ss)==1:
        return {'initials':[],'names':[],'lastnames':[ss[0]]}
    elif len(ss)==2:
        return {'initials':[ss[0][0]],'names':[ss[0]],'lastnames':[ss[1]]}
    elif len(ss)==3:
        if len(ss[1])==1:
            return {'initials':[ss[0][0],ss[1][0]],'names':ss[:2],'lastnames':ss[2:]}
        elif len(ss[0])==1:
            return {'initials':[ss[0][0]],'names':ss[:1],'lastnames':ss[1:]}

        else:
            return {'initials':[ss[0][0]],'names':ss[:1],'lastnames':ss[1:]}
    elif len(ss)==4:
        return {'initials':[ss[0][0],ss[1][0]],'names':ss[:2],'lastnames':ss[2:]}
    else:
        return {'initials':[ss[0][0],ss[1][0]],'names':ss[:2],'lastnames':ss[2:]}


In [750]:
pd.set_option('display.max_colwidth',200)

In [751]:
QUALITY_CHECK=False
if QUALITY_CHECK:
    df_not_pp_udea=pd.DataFrame(not_pp_udea)

In [753]:
#df_not_pp_udea[df_not_pp_udea.authorships.apply(lambda authorships: set([a.get('author').get('id') for a in authorships]
                                                        #).intersection(d.keys())).apply(len)>0].reset_index(drop=True)[150:].loc[159].concepts

In [754]:
aua=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vRXL8uTvlvkOh6fka8a7ezr7tpT2emjQWP9dhx1Ky8NvrEUJkkVn1svndL2PfdoD3CCEbmH7dz_gwtE/pub?output=xlsx')

In [755]:
aua[:2]

,Timestamp,Nombre,Email,Títulos,Grupo,Perfil en Google Scholar,Programas,Pensionado,CvLAC,email fisica,Nombres,Apellidos
0,NaT,Arenas Gaviria Bernardo,bernardo.arenas@udea.edu.co,"Físico, Magíster en Física.",NaN,No,Maestría,NaN,NaN,barena@fisica.udea.edu.co,Bernardo,Arenas Gaviria
1,NaT,Arnache Olmos Óscar L.,oscar.arnache@udea.edu.co,"Físico, Magister en Física, Doctor en Física.",GES: Estado Sólido,https://scholar.google.com/citations?user=R5OkApAAAAAJ&hl=en,Doctorado,NaN,http://scienti.colciencias.gov.co:8081/cvlac/visualizador/generarCurriculoCv.do?cod_rh=0000036048,oarnache@fisica.udea.edu.co,Óscar L.,Arnache Olmos


In [777]:
import unidecode as ud
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
def find_name(s,auam):
    aua=auam.copy()
    d={}
    try:
        aua['Nombres_reversed']=(aua['Nombres']+' '+aua['Apellidos'])
        us=ud.unidecode(s)
        eo=process.extractOne(us,  aua['Nombres_reversed'].apply(ud.unidecode) ,scorer=fuzz.ratio,score_cutoff=60)
        if len(set(split_names(us).get('initials')+split_names(us).get('lastnames')).intersection(
               set(split_names(eo[0]).get('initials')+split_names(eo[0]).get('lastnames'))))>1:
            row=aua[aua['Nombres_reversed'].apply(ud.unidecode)==eo[0]].iloc[0]
            d=split_names(row['Nombres_reversed']) 
            d['full_name']=row['Nombres_reversed']
            d['cod_rh']=row['CvLAC'].split('cod_rh=')[-1]
    except:
        pass
    return d

In [778]:
pp_udea_au_unique['full_name']=pp_udea_au_unique.display_name.apply(lambda s:find_name(s,aua))
#kk.iloc[[5]]

In [ ]:
pp_udea_au_unique[pp_udea_au_unique.full_name.apply(len)>0].iloc[[0]]

,author_position,author,institutions,raw_affiliation_string,id,display_name,works_count,full_name
55,middle,"{'id': 'https://openalex.org/A2967939840', 'display_name': 'N. Vanegas Arbelaez', 'orcid': None}","[{'id': 'https://openalex.org/I35961687', 'display_name': 'University of Antioquia', 'ror': 'https://ror.org/03bp5hc83', 'country_code': 'CO', 'type': 'education'}]",None,https://openalex.org/A2967939840,N. Vanegas Arbelaez,97,"{'initials': ['N'], 'names': ['Nelson'], 'lastnames': ['Vanegas', 'Arbeláez'], 'full_name': 'Nelson Vanegas Arbeláez', 'cod_rh': '0000037206'}"


In [854]:
for d in pp_udea:
    authorships=d.get('authorships')
    for a  in authorships:
        prf=pp_udea_au_unique[pp_udea_au_unique[ "id"] ==  a.get('author').get('id') ]
        if not prf.empty and len(prf['full_name'].iloc[0])>0:
            a.get('author').update( prf['full_name'].iloc[0] )

In [855]:
pd.DataFrame(pp_udea).authorships.str[-1].str['author'].str['full_name'].dropna().unique()

array(['Oscar Alberto Zapata Noreña', 'Daniel Esteban Jaramillo Arango',
       'Diego Alejandro Restrepo Quintero',
       'Leonardo Augusto Pachón Contreras', 'Antonio Enea Romano ',
       'William Alberto Ponce Gutiérrez', 'Boris Anghelo Rodríguez Rey',
       'Jorge Iván Zuluaga Callejas', 'Jesús María Mira Martínez'],
      dtype=object)

In [864]:
f=open('data/openalex_gfif.json','w')
json.dump(pp_udea,f)
f.close()

In [865]:
!gzip -f data/openalex_gfif.json 

In [868]:
pd.read_json('data/openalex_gfif.json.gz').shape

(340, 25)